In [3]:
import json
import os
import cv2
import matplotlib.pyplot as plt
import pickle
from random import shuffle

# Load files

In [ ]:
# load annotated_files
with open('/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/all/layout/dla_ls_20240924.json', 'r') as f:
    files = json.load(f)

file_names = [file["data"]["file_name"] for file in files]
for file in files:
    file['data']['file_name'] = file['data']['file_name'].replace('.pdf.pdf', '.pdf')
pdf_folder = os.listdir("/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/all/pdf_renamed")

"""
  To show that what files from annotated_files
  aren't existing in pdf_folder
"""

wrong_files = []
for name in file_names:
  if name not in pdf_folder:
    wrong_files.append(name)
# wrong_files.append("4470.pdf")
print(wrong_files)

In [ ]:
# with open('/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/qms_benchmark/layout/qms_annotated_layout.json') as f:
#     rule_based_coco = json.load(f)
# doc2id = {page['file_name']: i for i,page in enumerate(rule_based_coco['images'])}
with open('/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/doclaynet_9k/coco/train.json') as f:
    train = json.load(f)

with open('/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/doclaynet_9k/coco/val.json') as f:
    val = json.load(f)
with open('/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/doclaynet_9k/coco/test.json') as f:
    test = json.load(f)
with open('/home/vinhvq11/Desktop/PDF_extraction/fine_tuning_vgt/data_loader/qms_v2_benchmark/COCO/test.json') as f:
    qms=json.load(f)
with open('/home/vinhvq11/Desktop/PDF_extraction/fine_tuning_vgt/error_analysis/tpbank/filtered_groundtruth_coco.json') as f:
    tpbank=json.load(f)
img2set = {}
for img in train['images']:
    img2set[img['file_name']] = 'train'
for img in val['images']:
    img2set[img['file_name']] = 'val'
for img in test['images']:
    img2set[img['file_name']] = 'test'
for img in qms['images']:
    img2set[img['file_name']] = 'qms'
for img in tpbank['images']:
    img2set[img['file_name']] = 'tpbank'


In [ ]:
doc2id = {k.replace('.jpg','.png'): i for i,k in enumerate(img2set)}

In [36]:
img2set.values()

dict_values(['train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train', 'train

In [ ]:
len(doc2id)

In [4]:
with open('/home/vinhvq11/Desktop/PDF_extraction/data_process/extract_elements_text/fci/output/fci_ls_v3_20240930.json') as f:
    files = json.load(f)

In [5]:
page_id=0
doc2id = {}
for file in files:
    for doc in file['data']['document']:
        doc2id[os.path.basename(doc['page'].replace('.jpg','.png'))]= page_id
        page_id+=1

In [6]:
doc2id

{'Them_moi_mot_Credential_page_1.png': 0,
 'Them_moi_mot_Credential_page_2.png': 1,
 'Them_moi_mot_Credential_page_3.png': 2,
 'Them_moi_mot_Credential_page_4.png': 3,
 'Them_moi_mot_Credential_page_5.png': 4,
 'Them_moi_mot_Credential_page_6.png': 5,
 'Them_moi_mot_Credential_page_7.png': 6,
 'Them_moi_mot_Credential_page_8.png': 7,
 'Them_moi_mot_Credential_page_9.png': 8,
 'Them_moi_mot_Credential_page_10.png': 9,
 'Them_moi_mot_Credential_page_11.png': 10,
 '031eBMISM_Key_Custodian_Acknowledgement_Template_page_1.png': 11,
 'Quan_ly_Products_page_1.png': 12,
 'Quan_ly_Products_page_2.png': 13,
 'Quan_ly_Products_page_3.png': 14,
 'Quan_ly_Products_page_4.png': 15,
 'Quan_ly_Products_page_5.png': 16,
 'Quan_ly_Products_page_6.png': 17,
 'Quan_ly_Products_page_7.png': 18,
 'Quan_ly_Products_page_8.png': 19,
 'Quan_ly_Products_page_9.png': 20,
 '1_Tao_mot_Target_moi_page_1.png': 21,
 '1_Tao_mot_Target_moi_page_2.png': 22,
 '1_Tao_mot_Target_moi_page_3.png': 23,
 '1_Tao_mot_Target_moi_

# Mapping
LabelStudio format to DocLayNet format

In [7]:
convert_label_name = {
    "text": "Text",
    "title": "Section-header",
    "list-item": "List-item",
    "table": "Table",
    "figure": "Picture",
    "form": "Form",
    "footnote": "Footnote",
    "useless": "Useless",
    'page-header': 'Page-header',
    'page-footer': 'Page-footer'
}
annot_id=0
# categories
count = -1
categories = []
for key in convert_label_name:
    count += 1
    category = {}
    category["id"] = count; category["name"] = key
                                              #convert_label_name[key]
    categories.append(category)

map_category_to_id = {item["name"]: item["id"] for item in categories}

# write into json file
def write_info_to_json(files, start_id,set='train'):
    global annot_id
    label_counts = {l:0 for l in convert_label_name.keys()}
    json_file = {
        "images": [],
        "annotations": [],
        "categories": categories
    }
    page_id = start_id
    # annot_id = start_id
    for file in files:
        page_shape = {}
        # for page in file["annotations"][0]["result"]:
        #   page_shape[page["to_name"]] = (page["original_height"], page["original_width"]) #(height, width)
        pages = {}
        for page_dict in file["data"]["document"]:
            page_url = page_dict["page"]
            # if page_url not in page_names:
            #   continue
            page_info = {}
            file_page = page_url[page_url.find("page_"): page_url.find(".")] # ex: page_0
            page_idx = int(file_page[5:])-1
            file_page = f'page_{page_idx}'
            page_name = page_url.replace("/data/local-files/?d=documents/", "")
                                        # /data/local-files/?d=documents/
            page_name = page_name.replace(".jpg", ".png")
            # page info
            # if page_name not in img2set:
            #   continue
            page_info["id"] = doc2id[page_name]
            page_info["file_name"] = page_name
            # if img2set[page_name]!=set:
            #   continue
            if file_page in page_shape:
              page_info["height"] = page_shape[file_page][0]
              page_info["width"] = page_shape[file_page][1]
              json_file["images"].append(page_info)
            if os.path.exists(os.path.join("/home/vinhvq11/Desktop/PDF_extraction/data_process/extract_elements_text/fci/imgs", page_name)):
              img_shape = cv2.imread(os.path.join("/home/vinhvq11/Desktop/PDF_extraction/data_process/extract_elements_text/fci/imgs", \
                                                  page_name)).shape # (height, width, channel)
              page_info["height"] = img_shape[0]
              page_info["width"] = img_shape[1]
              json_file["images"].append(page_info)
            else:
              print(page_name)
            pages[file_page]=page_name
            page_id += 1
        for id, annotation in enumerate(file["annotations"][0]["result"]):
            if annotation["to_name"] in pages.keys():
              img_shape = None
              page_name = pages[annotation["to_name"]]

              if os.path.exists(os.path.join("/home/vinhvq11/Desktop/PDF_extraction/data_process/extract_elements_text/fci/imgs", page_name)):
                img_shape = cv2.imread(os.path.join("/home/vinhvq11/Desktop/PDF_extraction/data_process/extract_elements_text/fci/imgs", \
                                                    page_name)).shape # (height, width, channel)
              else:
                print(page_name)
                continue
              # print(page_name)
              # print(img_shape)
              # print(annotation)
              annot = {}
              x_min = annotation["value"]["x"]*img_shape[1]/100
              y_min = annotation["value"]["y"]*img_shape[0]/100
              width = annotation["value"]["width"]*img_shape[1]/100
              height = annotation["value"]["height"]*img_shape[0]/100

              bbox = [x_min, y_min, width, height]
              annot["bbox"] = bbox
              annot["bbox_mode"] = 1
              # annot['score'] =annotation['score']
              if "rectanglelabels" in annotation["value"]:
                  old_label = annotation["value"]["rectanglelabels"][0]
              else:
                  print(annotation)
                  continue
              annot["category_id"] = map_category_to_id[old_label.strip()]
              label_counts[old_label.strip()]+=1
                                      # map_category_to_id[convert_label_name[old_label.strip()]]
              annot["image_id"] = doc2id[page_name]
              annot["id"] = annot_id
              annot["iscrowd"] = 0
              annot["area"] = width * height

              json_file["annotations"].append(annot)
              annot_id += 1
            # else:
            #   print(annotation["to_name"])

    return json_file, label_counts

# Analyze pages/each domain in train/val:

In [8]:
label_count = {}
train_data, train_label_counts = write_info_to_json(files=files, start_id=0,set='train')
json.dump(train_data, open("/home/vinhvq11/Desktop/PDF_extraction/data_process/extract_elements_text/fci/output/gt_coco_fci_20240930.json", 'w'),ensure_ascii=False,indent=4)

In [1]:
train_label_counts

NameError: name 'train_label_counts' is not defined

In [35]:
# len(train_annotated_file)/len(annotated_file)
label_count = {}
train_data, train_label_counts = write_info_to_json(files=files, start_id=0,set='train')
json.dump(train_data, open("/home/vinhvq11/Desktop/PDF_extraction/fine_tuning_vgt/data_loader/DocLayNet_9k/coco_v1_20240924/train.json", 'w'),ensure_ascii=False,indent=4)
label_count['train'] = train_label_counts 
train_data, train_label_counts = write_info_to_json(files=files, start_id=10000,set='val')
json.dump(train_data, open("/home/vinhvq11/Desktop/PDF_extraction/fine_tuning_vgt/data_loader/DocLayNet_9k/coco_v1_20240924/val.json", 'w'),ensure_ascii=False,indent=4)
label_count['val'] = train_label_counts 
train_data, train_label_counts = write_info_to_json(files=files, start_id=20000,set='test')
json.dump(train_data, open("/home/vinhvq11/Desktop/PDF_extraction/fine_tuning_vgt/data_loader/DocLayNet_9k/coco_v1_20240924/test.json", 'w'),ensure_ascii=False,indent=4)
label_count['test'] = train_label_counts
json.dump(label_count, open("/home/vinhvq11/Desktop/PDF_extraction/fine_tuning_vgt/data_loader/DocLayNet_9k/coco_v1_20240924/label_count.json", 'w'),ensure_ascii=False,indent=4)

# val_data, val_label_counts = write_info_to_json(files=files,page_names = val_annotated_pages,start_id=10000)
# test_data, test_label_counts = write_info_to_json(files=files,page_names = test_annotated_files,start_id=20000)


{'original_width': 596, 'original_height': 842, 'image_rotation': 0, 'value': {'x': 50.13214750178294, 'y': 42.70673093077703, 'width': 1.2585476360280257, 'height': 2.225618350965174, 'rotation': 0}, 'id': 'q2sRYFDi5Z', 'from_name': 'labels_60', 'to_name': 'page_60', 'type': 'rectanglelabels', 'origin': 'manual'}


In [39]:
# len(train_annotated_file)/len(annotated_file)
with open("/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/doclaynet_9k/coco_v1_20240924/label_count.json", 'r') as f:
    label_count = json.load(f)
train_data, train_label_counts = write_info_to_json(files=files, start_id=0,set='tpbank')
json.dump(train_data, open("/home/vinhvq11/Desktop/PDF_extraction/fine_tuning_vgt/data_loader/DocLayNet_9k/coco_v1_20240924/tpbank_benchmark.json", 'w'),ensure_ascii=False,indent=4)
label_count['tpbank_benchmark'] = train_label_counts 
train_data, train_label_counts = write_info_to_json(files=files, start_id=0,set='qms')
json.dump(train_data, open("/home/vinhvq11/Desktop/PDF_extraction/fine_tuning_vgt/data_loader/DocLayNet_9k/coco_v1_20240924/qms_benchmark.json", 'w'),ensure_ascii=False,indent=4)
label_count['qms_benchmark'] = train_label_counts 
json.dump(label_count, open("/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/doclaynet_9k/coco_v1_20240924/label_count.json", 'w'),ensure_ascii=False,indent=4)


2023_05_202305251754299329PM_1_page_0.png
2023_05_202305251754299329PM_1_page_1.png
2023_05_202305251754299329PM_1_page_2.png
2023_05_202305251754299329PM_1_page_3.png
2023_05_202305251754299329PM_1_page_4.png
2023_05_202305251754299329PM_1_page_5.png
2023_05_202305251754299329PM_1_page_6.png
2023_05_202305251754299329PM_1_page_7.png
2023_05_202305251754299329PM_1_page_8.png
2023_05_202305251754299329PM_1_page_9.png
2023_05_202305251754299329PM_1_page_10.png
2023_05_202305251754299329PM_1_page_11.png
2023_05_202305251754299329PM_1_page_12.png
2023_05_202305251754299329PM_1_page_13.png
2023_05_202305251754299329PM_1_page_14.png
2023_05_202305251754299329PM_1_page_15.png
2023_05_202305251754299329PM_1_page_16.png
2023_05_202305251754299329PM_1_page_17.png
2023_05_202305251754299329PM_1_page_18.png
2023_05_202305251754299329PM_1_page_19.png
2023_05_202305251754299329PM_1_page_20.png
2023_05_202305251754299329PM_1_page_21.png
2023_05_202305251754299329PM_1_page_1.png
2023_05_20230525175429

In [ ]:
json.dump(label_count, open("/home/vinhvq11/Desktop/PDF_extraction/fine_tuning_vgt/data_loader/DocLayNet_9k/coco_v1_20240910/label_count.json", 'w'),ensure_ascii=False,indent=4)


In [ ]:
import pandas as pd
import json
with open('/home/vinhvq11/Desktop/PDF_extraction/fine_tuning_vgt/data_loader/DocLayNet_9k/coco_v1_20240910/label_count.json', 'r') as f:
    label_count = json.load(f)
df = pd.DataFrame(label_count).T
df.to_excel('/home/vinhvq11/Desktop/PDF_extraction/fine_tuning_vgt/data_loader/DocLayNet_9k/coco_v1_20240910/label_count.xlsx')

In [ ]:
img2set = {}
for domain in os.listdir('/home/vinhvq11/Desktop/PDF_extraction/fine_tuning_vgt/data_loader/DocLayNet_9k/coco_v1_20240910/test_domain'):
    
    domain_name = domain.replace('.json', '')
    with open(f'/home/vinhvq11/Desktop/PDF_extraction/fine_tuning_vgt/data_loader/DocLayNet_9k/coco_v1_20240910/test_domain/{domain}') as f:
        coco_ds=json.load(f)
    print(len(coco_ds['images']))
    for img in coco_ds['images']:
        
        img2set[img['file_name']] = domain_name
    with open("/home/vinhvq11/Desktop/PDF_extraction/fine_tuning_vgt/data_loader/DocLayNet_9k/coco_v1_20240910/label_count.json", 'r') as f:
        label_count = json.load(f)
    train_data, train_label_counts = write_info_to_json(files=files, start_id=0,set=domain_name)
    json.dump(train_data, open(f"/home/vinhvq11/Desktop/PDF_extraction/fine_tuning_vgt/data_loader/DocLayNet_9k/coco_v1_20240910/test_domain/{domain}", 'w'),ensure_ascii=False,indent=4)
    label_count[domain_name] = train_label_counts 
    json.dump(label_count, open("/home/vinhvq11/Desktop/PDF_extraction/fine_tuning_vgt/data_loader/DocLayNet_9k/coco_v1_20240910/label_count.json", 'w'),ensure_ascii=False,indent=4)


In [ ]:
img2set.keys()

In [ ]:
len(img2set['test_law'])

In [ ]:
json.dump(train_data, open("train_ls.json", 'w'))

In [ ]:
print('train: ',train_label_counts)
# print('val: ',val_label_counts)
# print('test: ',test_label_counts)

In [ ]:

list_png = os.listdir("./DocLayNet/PNG/")

def object(data):
    object = {key: [] for key in train_data}
    ids = []

    for image in data["images"]:
        if image["file_name"] != "11353_CV_TRIEN_KHAI_NANG_CAP_SMARTBANKING_LAN_41_NAM_2022_page_3.png":
          object["images"].append(image)
          ids.append(image["id"])

    for annot in data["annotations"]:
        if annot["image_id"] in ids:
            object["annotations"].append(annot)
    object["categories"] = data["categories"]

    for id, image in enumerate(data["images"]):
      if "11353_CV_TRIEN_KHAI_NANG_CAP_SMARTBANKING_LAN_41_NAM_2022" in image["file_name"]:
          image_shape = cv2.imread("./DocLayNet/PNG/" + image["file_name"]).shape
          object["images"][id]["height"] = image_shape[0]
          object["images"][id]["width"] = image_shape[1]

    return object

train_data = object(train_data)
# val_data = object(val_data)
# test_data = object(test_data)

train_object = json.dumps(train_data)
# val_object = json.dumps(val_data)
# test_object = json.dumps(test_data)
# with open("./DocLayNet/COCO/train.json", "w") as file:
#     file.write(train_object)

# with open("./DocLayNet/COCO/val.json", "w") as file:
#     file.write(val_object)
with open("/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/tpbank_benchmark/model_predictions.json", "w") as file:
    file.write(train_object)

In [34]:
print(len(json.load(open("/home/vinhvq11/Desktop/PDF_extraction/fine_tuning_vgt/data_loader/DocLayNet_9k/coco_v1_20240924/train.json"))['images']))
print(len(json.load(open("/home/vinhvq11/Desktop/PDF_extraction/fine_tuning_vgt/data_loader/DocLayNet_9k/coco_v1_20240924/val.json"))['images']))
print(len(json.load(open("/home/vinhvq11/Desktop/PDF_extraction/fine_tuning_vgt/data_loader/DocLayNet_9k/coco_v1_20240924/test.json"))['images']))
# val_data, val_label_counts = write_info_to_json(files=files,page_names = val_annotated_pages,start_id=10000)
# test_data, test_label_counts = write_info_to_json(files=files,page_names = test_annotated_files,start_id=20000)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)